
# Distributed CNTK using custom docker images
In this tutorial, you will train a CNTK model on the MNIST dataset using a custom docker image and distributed training.



In [ ]:
%reload_azureml_ws

# 1. Create or Attach existing AmlCompute

You will need to create a compute target for training your model. In this tutorial, you create AmlCompute as your training compute resource.

Creation of AmlCompute takes approximately 5 minutes. If the AmlCompute with that name is already in your workspace this code will skip the creation process.

As with other Azure services, there are limits on certain resources (e.g. AmlCompute) associated with the Azure Machine Learning service. Please read this article on the default limits and how to request more quota.

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cluster_name = "gpu-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6',
                                                           max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True)

# use get_status() to get a detailed status for the current AmlCompute
print(compute_target.get_status().serialize())

# 2. Upload training data

For this tutorial, we will be using the MNIST dataset.

First, let's download the dataset. We've included the install_mnist.py script to download the data and convert it to a CNTK-supported format. Our data files will get written to a directory named 'mnist'.

In [ ]:
from src.azureds_util.nodes.cntk_distr_mnist import install_mnist

install_mnist.main('data/01_raw/mnist')


To make the data accessible for remote training, you will need to upload the data from your local machine to the cloud. AML provides a convenient way to do so via a Datastore. The datastore provides a mechanism for you to upload/download data, and interact with it from your remote compute targets.

Each workspace is associated with a default datastore. In this tutorial, we will upload the training data to this default datastore, which we will then mount on the remote compute for training in the next section.

In [ ]:
ds = ws.get_default_datastore()
print(ds.datastore_type, ds.account_name, ds.container_name)

The following code will upload the training data to the path ./mnist on the default datastore.

In [ ]:
ds.upload(src_dir='data/01_raw/mnist', target_path='./01_raw/mnist')


Now let's get a reference to the path on the datastore with the training data. We can do so using the path method. In the next section, we can then pass this reference to our training script's --data_dir argument.



In [ ]:
path_on_datastore = '01_raw/mnist'
ds_data = ds.path(path_on_datastore)
print(ds_data)

# 3.  Train model on the remote compute¶

Now that we have the cluster ready to go, let's run our distributed training job.

## 3.1 Create an experiment
Create an experiment to track all the runs in your workspace for this distributed CNTK tutorial.

In [ ]:
from azureml.core import Experiment

experiment_name = 'cntk-distr'
experiment = Experiment(ws, name=experiment_name)

## 3.2 Create an Estimator

The AML SDK's base Estimator enables you to easily submit custom scripts for both single-node and distributed runs. You should this generic estimator for training code using frameworks such as sklearn or CNTK that don't have corresponding custom estimators. For more information on using the generic estimator, refer [here](https://docs.microsoft.com/en-gb/azure/machine-learning/how-to-train-ml-models).

In [ ]:

from azureml.train.estimator import Estimator

script_params = {
    '--num_epochs': 20,
    '--data_dir': ds_data.as_mount(),
    '--output_dir': './outputs'
}

estimator = Estimator(source_directory='src/azureds_util/nodes/cntk_distr_mnist',
                      compute_target=compute_target,
                      entry_script='cntk_distr_mnist.py',
                      script_params=script_params,
                      node_count=2,
                      process_count_per_node=1,
                      distributed_backend='mpi',
                      pip_packages=['cntk-gpu==2.6'],
                      custom_docker_image='microsoft/mmlspark:gpu-0.12',
                      use_gpu=True)

We would like to train our model using a pre-built Docker container. To do so, specify the name of the docker image to the argument custom_docker_image. Finally, we provide the cntk package to pip_packages to install CNTK 2.6 on our custom image.

The above code specifies that we will run our training script on 2 nodes, with one worker per node. In order to run distributed CNTK, which uses MPI, you must provide the argument distributed_backend='mpi'.

# 3.3 Submit job
Run your experiment by submitting your estimator object. Note that this call is asynchronous.


In [ ]:
run = experiment.submit(estimator)
print(run)

# 3.4 Monitor your run
You can monitor the progress of the run with a Jupyter widget. Like the run submission, the widget is asynchronous and provides live updates every 10-15 seconds until the job completes.

In [ ]:
from azureml.widgets import RunDetails

RunDetails(run).show()

Alternatively, you can block until the script has completed training before running more code.



In [ ]:
run.wait_for_completion(show_output=True)